In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn


In [2]:
num_classes = 12
input_channels = 9
device = 'cpu'
win_size = 64
step = 32

In [3]:
def sliding_window(sequence,
                   win_size = 20, step = 5):
    nb_chunks = ((len(sequence)-win_size)//step)+1
    for i in range(0,nb_chunks*step,step):
        yield sequence[i:i+win_size]


In [4]:
new_col_names = ["time", "hand_accelerometer", "hand_magnetometer", "hand_gyroscope", 
                 "ankle_accelerometer", "ankle_magnetometer", "ankle_gyroscope", 
                 "chest_accelerometer","chest_magnetometer", "chest_gyroscope"]
replaced_cols = ["time", "hand_accelerometer", "hand_gyroscope", "hand_magnetometer", 
                 "ankle_accelerometer", "ankle_gyroscope","ankle_magnetometer",  
                 "chest_accelerometer", "chest_gyroscope","chest_magnetometer"]
new_col_names = new_col_names[:input_channels//3+1]
replaced_cols = replaced_cols[:input_channels//3+1]

In [5]:
replaced_cols

['time', 'hand_accelerometer', 'hand_gyroscope', 'hand_magnetometer']

In [6]:
def transform_csv(path):
    
    
    
    def column_transform(data,col_name):
        if col_name != "time":
            new = data[col_name].str.replace(")","").str.replace("(","").str.replace(" ","").str.split(",", n = 2, expand = True) 
            data[col_name + "_x"] = new[0]
            data[col_name + "_y"] = new[1]
            data[col_name + "_z"] = new[2]
            data.drop(columns =[col_name], inplace = True) 
        return data
    
    data = pd.read_csv(path)
    
    data = data.rename(index=str,columns = dict(zip(list(data.columns),new_col_names)))
    
    data = data[replaced_cols]
    


    for col in data.columns:
        data = column_transform(data, col)
    return data

In [7]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [8]:
demo_activities4 = "lying,sitting,standing,walking".split(',')

demo_activities12 = ['lying',
 'sitting',
 'standing',
 'walking',
 'running',
 'cycling',
 'Nordic walking',
 'ascending stairs',
 'descending stairs',
 'vacuum cleaning',
 'ironing',
 'rope jumping']


In [9]:
demo_mode =  True

In [10]:
activities = demo_activities12

In [11]:
256*(win_size//(2**3))

2048

In [12]:
class Conv1DNet(nn.Module):
    
    def __init__(self):
        super(self.__class__, self).__init__()
        self.input_bn = nn.BatchNorm1d(input_channels)
        self.conv11 = nn.Conv1d(in_channels=input_channels, out_channels=64, kernel_size=3, padding=1)
        self.pool11 = nn.MaxPool1d(kernel_size=2)
        self.conv21 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool21 = nn.MaxPool1d(kernel_size=2)
        self.conv31 = nn.Conv1d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.pool31 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(256*(win_size//(2**3)),512)
        self.fc2 = nn.Linear(512,512)
        self.logit = nn.Linear(512,num_classes)
        
        self.features = nn.Sequential(
             self.input_bn,
             self.conv11,self.pool11,
             nn.ReLU(),                            
             self.conv21,self.pool21,
             nn.ReLU(),
             self.conv31,self.pool31,
             nn.ReLU(),
             Flatten(),
        )
        
        self.classifier = nn.Sequential(
             self.fc1,
             nn.ReLU(),
             self.fc2,
             nn.ReLU(),
             self.logit,
        )
        
    def forward(self,input):
        features = self.features(input)
        print(features.shape)
        return self.classifier(features)

In [13]:
sPATH = '/ksozykinraid/har-models/Conv1DNet_onehand_12class_winsize64_0.1037.pth'

In [14]:
model = Conv1DNet().to(device)

In [15]:
dummy_batch = torch.rand(10,win_size,input_channels).transpose(2,1)

In [16]:
model(dummy_batch)

torch.Size([10, 2048])


tensor([[ 0.0313, -0.0342,  0.0144,  0.0067, -0.0152,  0.0175, -0.0275, -0.0439,
         -0.0350,  0.0505,  0.0020,  0.0464],
        [ 0.0292, -0.0307,  0.0156,  0.0054, -0.0119,  0.0211, -0.0297, -0.0484,
         -0.0390,  0.0464,  0.0085,  0.0441],
        [ 0.0272, -0.0305,  0.0154,  0.0054, -0.0123,  0.0195, -0.0251, -0.0459,
         -0.0427,  0.0445,  0.0065,  0.0460],
        [ 0.0242, -0.0271,  0.0152,  0.0006, -0.0085,  0.0244, -0.0253, -0.0472,
         -0.0402,  0.0406,  0.0056,  0.0426],
        [ 0.0258, -0.0333,  0.0150,  0.0056, -0.0112,  0.0207, -0.0252, -0.0445,
         -0.0390,  0.0450,  0.0035,  0.0491],
        [ 0.0272, -0.0311,  0.0150,  0.0040, -0.0140,  0.0247, -0.0259, -0.0487,
         -0.0415,  0.0491,  0.0064,  0.0450],
        [ 0.0300, -0.0296,  0.0173,  0.0055, -0.0144,  0.0192, -0.0286, -0.0444,
         -0.0373,  0.0473,  0.0065,  0.0451],
        [ 0.0236, -0.0292,  0.0149,  0.0024, -0.0118,  0.0212, -0.0279, -0.0499,
         -0.0383,  0.0442,  0.

In [17]:
model.load_state_dict(torch.load(sPATH));
model.eval();

In [26]:
data = transform_csv('/ksozykinraid/data/sensors/real/sitting.csv')
data = data.drop(['time'],1)
data = data.values.astype('float32')

In [27]:
batch = []
model.eval()
for window in sliding_window(data,win_size=win_size,step=16):
    batch.append(window)
batch = torch.tensor(np.array(batch)).to(device)
print(batch.shape)
batch = batch.transpose(2,1)

torch.Size([39, 64, 9])


In [28]:
probas = model(batch).softmax(-1)
preds = torch.argmax(probas,-1)
txt_preds = [activities[int(e)] for e in torch.argmax(model(batch).softmax(-1),-1)]
classes, counts = np.unique(txt_preds,return_counts=True)

torch.Size([39, 2048])
torch.Size([39, 2048])


In [29]:
final_predcit = classes[np.argmax(txt_preds)]

In [30]:
final_predcit

'walking'

In [ ]:
import bluepy
from bluepy import sensortag

import sys
import traceback
import struct 

acceler = []
gyroscope = []
magno = []

mac = 'MAC ADRESS'

tag = None

print('connecting to '+mac, end='', flush=True)
while tag == None:
    try:
        tag = sensortag.SensorTag(mac)
    except (bluepy.btle.BTLEException):
        print('.', end='', flush=True)
    except (Excpetion, e):
        print()
        print(e)
        sys.exit()

tag.accelerometer.enable()
tag.magnetometer.enable()
tag.gyroscope.enable()

while True:
    acceler.append(tag.accelerometer.read())
    gyroscope.append(tag.gyroscope.read())
    magno.append(tag.magnetometer.read() )
    
    if len(acceler) == 64:
        batch = []
        data = np.stack((acceler, gyroscope, magno), axis= - 1)
        model.eval()
        for window in sliding_window(data,win_size=win_size,step=16):
            batch.append(window)
        batch = torch.tensor(np.array(batch)).to(device)
        #print(batch.shape)
        batch = batch.transpose(2,1)
        
        probas = model(batch).softmax(-1)
        preds = torch.argmax(probas,-1)
        txt_preds = [activities[int(e)] for e in preds]
        classes, counts = np.unique(txt_preds,return_counts=True)
        final_predcit = classes[np.argmax(counts)]
        
        print('final_predcit =', final_predcit)
        
        
        del acceler 
        del gyroscope 
        del magno 
        acceler = []
        gyroscope = []
        magno = []
        
    tag.waitForNotifications(0.1)
        
        